In [ ]:
options(repr.matrix.max.cols=10, repr.matrix.max.rows=10)
options(repr.plot.width=22, repr.plot.height=22)
setwd('/home/peter/fstats_tutorial')
suppressPackageStartupMessages({
    library(admixtools)
    library(tidyverse)
    source("/home/peter/fstats_tutorial/scripts/analysis.R")
})

# 0. Resources
- [`Admixtools`](https://github.com/DReichLab/AdmixTools)
- [`Admixtools 2`](https://github.com/uqrmaie1/admixtools)
- [`admixr`](https://github.com/bodkan/admixr)
- [Patterson et al. (2012)](http://www.genetics.org/content/192/3/1065)
- [Bhatia et al. (2013)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3759727/)
- [Peter (2016)](http://www.genetics.org/content/202/4/1485)
- [Petr et al. (2019)](https://academic.oup.com/bioinformatics/advance-article/doi/10.1093/bioinformatics/btz030/5298728)


# 1.  Setting up
To get your own copy of this tutorial, you can copy it on the server from my folder:

```bash
rsync -av /home/peter/fstats_tutorial . --exclude data
```

If you want the raw data as well, remove the `--exclude` flag

We will use the `R`-package `admixtools` for computation. This package is a fast implementation of $F$-statistics that works well if there is little missing data. It allows for precomputation of statistics, which saves us a lot of time. For this tutorial, I already prepared the data; but it can easily be regenerated using the following lines of code.

In [ ]:
#admixtools::extract_f2('data/world/worldfoci2', outdir='fdata/worldfoci2', blgsize = 0.05)
#admixtools::extract_f2('data/world/ancient', outdir='fdata/ancient', blgsize = 0.05)
#admixtools::extract_f2('data/europe/westeurasian1', outdir='fdata/westeurasia1', blgsize = 0.05)
#admixtools::extract_f2('data/europe/westeurasian2', outdir='fdata/westeurasia2', blgsize = 0.05)

# 2. Data
For this tutorial, we will be using data from the ancient DNA compendium by David Reich's lab,  that can be downloaded from https://reich.hms.harvard.edu/allen-ancient-dna-resource-aadr-downloadable-genotypes-present-day-and-ancient-dna-data. This data has an unique ascertainment
scheme, so throughout we need to keep ascertainment bias in mind. 

In order to save time, I subset the data and pre-computed F2-statistics that we will be using throughout. We will be using two distinct data sets; one focused on Western Eurasian diversity (`europe`), one representing global human diversity (`world`), and one designed to investigate archaic ancestry (`ancient`):

In [ ]:
list.files("/home/peter/fstats_tutorial/data/europe/");
list.files("/home/peter/fstats_tutorial/data/world/");
list.files("/home/peter/fstats_tutorial/data/ancient/");

The `.ind` and `.snp`-files are text files with information about each individual and each SNP respectively. The `.geno`-file is a compressed file with the actuall genotype information. See [this file](https://github.com/DReichLab/AdmixTools/blob/master/convertf/README) for detailed information on the file format. The text files can be read into R using the standard i/o functions:

In [ ]:
ind_world <- readr::read_table("/home/peter/fstats_tutorial/data/world/worldfoci2.ind", col_names=c("ind", "sex", "pop"), col_types='ccc')
ind_eu <- readr::read_table("/home/peter/fstats_tutorial/data/europe//westeurasian1.ind", col_names=c("ind", "sex", "pop"), col_types='ccc')
ind_ancient <- readr::read_table("/home/peter/fstats_tutorial/data/world/ancient.ind", col_names=c("ind", "sex", "pop"), col_types='ccc')

ind_eu

**Question 1:** How many individuals and populations are there in these data sets?

### 2.1 calculating FST
In general, there is a very large number of different estimators of $F_{ST}$ (see [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3759727/) for details). Here, we will use Hudson's estimator which is relatively simple and robust. We can calculate all pairwise $F_{ST}$-values using the `fst` function of `admixtools`:

In [ ]:
f2s_eu = read_f2("/home/peter/fstats_tutorial/fdata/westeurasia1") #load precomputed data
f2s_world = read_f2("/home/peter/fstats_tutorial/fdata/worldfoci2") #load precomputed data

pwfst1=fst(f2s_eu)
pwfst2=fst(f2s_world)

In [ ]:
pwfst2

These tables give us the pairwise $F_{ST}$-values between all pairs of populations. Are these values big or small? This is quite difficult to answer without context, as it will depend on the type of data you are analyzing, the amount of data and the scope of your study. To provide context, one often looks at a matrix of $F_{ST}$-values, which can be visualized using a heatmap. We are going to use the function `fst_plot` that I wrote for this purpose. It will transform above data frames into a matrix, and then generate a heatmap using the `heatmap.2`-function. In case you are interested in details, you can see the function definition by just typing `fst_plot` on the command line.

In [ ]:
fst_plot(f2s_eu, cex.label=0.8) #if not all labels are plotted on your screen, you might need to adjust the label size

**Task 1**: Create a $F_{ST}$-plot for the world data set using the same function as above. How do the results compare between the two data sets?

A pairwise matrix of $F_{ST}$ or $F_2$-values yields a basic overview of a data set from a structured population. It is the basis for many follow-up applications, including building trees and admixture graphs (tomorrow) or computing a PCA.

## 3. Admixture-F3
We will use the `world`-dataset that contains a small set of samples from all
continents. We'll hypothesize that African Americans (`AA`) are an admixed group resulting from the mixture of two populations. We'll assume that one source population are Yoruba from Nigeria, and 

1. aim to test our hypothesis that African Americans are admixed
2. make some inference what the other source population may be

In [ ]:
f3_aa = f3(f2s_world, pop1='AA', pop2='Yoruba', pop3=unique(ind_world$pop)) %>% arrange(est)
f3_aa

In [ ]:
f3_aa %>% 
    mutate(is_significant=z < -3, pop3=fct_reorder(pop3, -est)) %>% 
    ggplot(aes(x=pop3, y=est, ymin=est-3*se, ymax = est+3*se, color=is_significant)) +
    geom_point() + geom_errorbar() +geom_hline(yintercept = 0) + coord_flip() + theme_bw(25) +
    ggtitle("f3(AA; Yoruba, X)") + xlab(NULL) + ylab("f3")

**Question 2**: 
- Are any $F_3$-statistics negative? 
- Are they significant? (|Z| > 3?)?
- Which pairs of populations do these correspond to? 
- What could this mean about the population genetic history of the African Americans?

## 4. Outgroup-F3
Next, we can compute outgroup-F3 stats in a very similar fashion. We use outgroup-$F_3$-statistics of the form $F_3(\text{Outgroup}; \text{Unknown}, X_i)$ to find the population $X_i$ that is closest to the unknown population. For the next task, we will calculate some outgroup-$F_3$-statistics. We'll use the Western Eurasian data, which I modified to add an Outgroup (`Mbuti`, a population from central Africa with minimal European ancestry). and an  additional population labelled as `Unknown`. 

Which population could they be most closely related to?

We can use a very similar approach as before:
 - pop1 will be the `Mbuti`-population
 - pop2 will be the `Unknown`-population
 - pop3 will be the list of all candidates

**Task 2:** Use `admixtools`  to 
1. load the `fdata/westeurasian2`-data set into memory using the `admixtools::read_f2`-function
2. read all populations present in that data into memory
3. calculate all outgroup-$F_3$-statistics
4. create a figure visualizing your results, adapting the figure from above

In [ ]:
viz_outgroup_f3 <- function(f3_res){
    f3_res %>% mutate(pop3=fct_reorder(pop3, est)) %>%  
    ggplot(aes(x=pop3, y=est, ymin=est-3*se, ymax = est+3*se)) +
    geom_point() + 
    geom_errorbar() +
    #geom_hline(yintercept = 0) + 
    coord_flip() + 
    theme_bw(25) +
    xlab(NULL) + ylab("f3")
    }

## 5. f4/D-statistics
next, we can test for admixture in the past history of humans by lookin for an excess of ABBA or an excess of BABA sites in a 4-population tree.
For this purpose, we use the ancient dataset, which contains a subset of the Chimpanzee, three Neandertals and a small number of modern Human population samples.
To calculate $F_4$-stats is very similar to calculating $F_3$: instead of `admixtools::f3` we use `admixtools::f4`:

**Task 3**: Use admixtools2  to
 - calculate all f4-statistics of the form $D(\text{African}, \text{Non-African}, \text{Neandertals}, \text{Primate})$ (the African populations are Mbuti and Yoruba; the Non-Africans are French, Han and Papuan)
 - create a figure visualizing your results (you may use the function below)

**Question 3**: Are any of the statistics significant? In what direction? Is this consistent with what you might expect? 

In [ ]:
viz_d <- function(d_res) 
    d_res %>%
    mutate(sig=as.factor(abs(z)>3)) %>%
    ggplot(aes(x=pop1, y=est, ymin=est-3*se, ymax=est+3*se, color=pop3,group=pop3)) + 
    geom_point(position=position_dodge(0.8)) + 
    geom_errorbar(position=position_dodge(0.8)) +
    geom_hline(yintercept=0) +
    theme_bw(25) +
    facet_grid(pop2~.)

In [ ]:
ancient = read_f2('/home/peter/fstats_tutorial/fdata/ancient')
pops = dimnames(ancient)[1][[1]]
pops

## 6. estimating admixture proportions
The final task in the F-statistics framework will be to calculate admixture proportions using the $F_4$-ratio. 

For this purpose, we will use a data set that aims at estimating the proportion of Neandertal ancestry in modern humanms, based on the analysis by [Petr et al. 2019](https://www.pnas.org/content/116/5/1639), who used the assumption that all Neanderthal ancestry is closer to a Neandertal from Vindija Cave (in Croatia) than a Neandertal genome from Denisova cave (Altai, Russia). 

We assume that the populations are related like this:
 <img src="F4.large.jpg" alt="F4-ratio" style="width: 400px;"/>
 
 and estimate the admixture proportion $\alpha$ as
 $$\alpha = \frac{f_4(A, O; X, C)}{f_4(A, O; B, C)}$$
   

**Task 3**: Use the `qpf4ratio`-function in `admixtools` to use the ancient data (that we used for the $D$-stats exercise to estimate the Neandertal ancestry in French, Han and Papuans.

- have a look at `?qpf4ratio` to figure out the arguments this function expects
- which population is in which position of the arguments? Construct a vector `v` or data frame that matches this order
- run the f4-ratio-estimation using `qpf4ratio(ancient, v)
- do the archaic ancestries differ between the three populations?